---
title: "Connecting a paired t-test with a Bayesian multilevel model (DRAFT)"
toc: true
toc_sticky: true
---

In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pymc3 as pm
import scipy.stats as stats
import seaborn as sns
from scipy.special import expit as logistic
import daft
from causalgraphicalmodels import CausalGraphicalModel

from scipy.optimize import curve_fit

In [2]:
%load_ext nb_black
%config InlineBackend.figure_format = 'retina'
%load_ext watermark
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")
az.rcParams["stats.hdi_prob"] = 0.89  # sets default credible interval used by arviz

<IPython.core.display.Javascript object>

In [3]:
sns.set_context("talk")

<IPython.core.display.Javascript object>

In [4]:
def standardize(x):
    x = (x - np.mean(x)) / np.std(x)
    return x

<IPython.core.display.Javascript object>

# Dataset, extrapolated from Wikipedia

Example taken [here](https://en.wikipedia.org/wiki/Student%27s_t-test#Dependent_t-test_for_paired_samples).

In [6]:
df_tests = pd.DataFrame(
    {
        "Name": ["Mike", "Melanie", "Melissa", "Mitchell"],
        "Test_1": [35, 50, 90, 78],
        "Test_2": [67, 46, 86, 91],
    }
)

<IPython.core.display.Javascript object>

In [88]:
df_tests

,Name,Test_1,Test_2,diff
0,Mike,35,67,-32
1,Melanie,50,46,4
2,Melissa,90,86,4
3,Mitchell,78,91,-13


<IPython.core.display.Javascript object>

Interestingly, the table in this example is labeled as an "example of repeated measures." Interestingly, in the Chapter 1 introduction and in Chapter 13, McElreath highlights multi-level models as a way to get "improved estimates for repeat sampling".

# T-statistic using scipy.stats

In [11]:
stats.ttest_rel(df_tests["Test_1"], df_tests["Test_2"])

Ttest_relResult(statistic=-1.0784834690588145, pvalue=0.3598054860063756)

<IPython.core.display.Javascript object>

# Calculate t-statistic by hand

Why? Cause we're weird like that.

$$t = \frac{\bar{X}_D - \mu_0}{s_D / \sqrt{n}}$$

where \(\bar{X}_D\) is the average of the differences between all pairs and \(s_D\) is the standard deviation of the differences. We can set \(mu_0\) to 0 if we want to test whether the average is significantly different. This is straightforward to calculate.

In [12]:
df_tests["diff"] = df_tests["Test_1"] - df_tests["Test_2"]

<IPython.core.display.Javascript object>

In [13]:
t_stat = df_tests["diff"].mean() / (df_tests["diff"].std() / np.sqrt(len(df_tests)))
print("Manual calculated t-statistic: ", t_stat)

Manual calculated t-statistic:  -1.0784834690588145


<IPython.core.display.Javascript object>

We get exactly the same answer as `scipy.stats` so that is comforting.

# Calculated with a Bayesian approach

Now the fun part. We can set this up as a multi-level linear model but let's start with a non multi-level model.

I'm going to treat each person as their own "cluster". The test is the repeated measure, with time as a categorical variable as indicated [here](https://www.researchgate.net/post/Paired-t-test-or-liner-mixed-model).

First, let's do some table reformatting so that we can use it in our model.

In [108]:
df_tests2 = pd.melt(
    df_tests.drop("diff", axis=1),
    id_vars="Name",
)
df_tests2.columns = ["Name", "Time", "Score"]
df_tests2["Name_code"] = pd.Categorical(df_tests2["Name"]).codes
df_tests2["Time_code"] = pd.Categorical(df_tests2["Time"]).codes
df_tests2["Score_std"] = standardize(df_tests2["Score"])
df_tests2

,Name,Time,Score,Name_code,Time_code,Score_std
0,Mike,Test_1,35,2,0,-1.610167
1,Melanie,Test_1,50,0,0,-0.875490
2,Melissa,Test_1,90,1,0,1.083649
3,Mitchell,Test_1,78,3,0,0.495907
4,Mike,Test_2,67,2,1,-0.042856
5,Melanie,Test_2,46,0,1,-1.071404
6,Melissa,Test_2,86,1,1,0.887735
7,Mitchell,Test_2,91,3,1,1.132627


<IPython.core.display.Javascript object>

## Non-multi-level model

$$\text{score}_i \text{ ~ Normal}(\mu_i, \sigma) $$
$$\mu_i = \alpha_{\text{SUBJECT}} + \gamma_{\text{TIME}} $$
$$\alpha_j \text{ ∼ Normal}(0, 1.5) \tag{regularizing prior}$$
$$\gamma_j \text{ ∼ Normal}(0, 1.5) \tag{regularizing prior}$$
$$\sigma \text{ ~ Exp}(1)$$

In [114]:
df_tests2

,Name,Time,Score,Name_code,Time_code,Score_std
0,Mike,Test_1,35,2,0,-1.610167
1,Melanie,Test_1,50,0,0,-0.875490
2,Melissa,Test_1,90,1,0,1.083649
3,Mitchell,Test_1,78,3,0,0.495907
4,Mike,Test_2,67,2,1,-0.042856
5,Melanie,Test_2,46,0,1,-1.071404
6,Melissa,Test_2,86,1,1,0.887735
7,Mitchell,Test_2,91,3,1,1.132627


<IPython.core.display.Javascript object>

In [132]:
with pm.Model() as m1:

    # prior for SD of testers
    sigma = pm.Exponential("sigma", 1.0)

    # regularizing prior
    a = pm.Normal("a", 0, 1.5, shape=4)  # one for each subject
    g = pm.Normal("g", 0, 1.5, shape=2)  # one for each time point

    # mu is deterministic, equivalent to alpha indexed by time
    mu = a[df_tests2["Name_code"]] + g[df_tests2["Time_code"]]

    # likelihood
    score = pm.Normal("score", mu=mu, sd=sigma, observed=df_tests2["Score_std"])

    trace_m1 = pm.sample(
        draws=1000, random_seed=19, return_inferencedata=True, progressbar=False
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [g, a, sigma]
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 22 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


<IPython.core.display.Javascript object>

In [126]:
az.summary(trace_m1)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a[0],-0.918,0.738,-2.022,0.244,0.030,0.021,615.0,615.0,619.0,1418.0,1.01
a[1],0.838,0.727,-0.323,1.989,0.029,0.020,649.0,649.0,650.0,1547.0,1.01
a[2],-0.782,0.744,-1.950,0.379,0.029,0.021,645.0,645.0,648.0,1662.0,1.01
a[3],0.689,0.728,-0.465,1.837,0.028,0.020,688.0,688.0,687.0,1464.0,1.01
g[0],-0.165,0.674,-1.175,0.967,0.028,0.020,560.0,560.0,566.0,1216.0,1.01
g[1],0.258,0.670,-0.838,1.266,0.029,0.021,532.0,532.0,538.0,1256.0,1.01
sigma,0.711,0.310,0.288,1.074,0.012,0.008,707.0,707.0,350.0,106.0,1.01


<IPython.core.display.Javascript object>

In [127]:
trace_m1_df = trace_m1.to_dataframe()
trace_m1_df.head()

,chain,draw,"(posterior, a[0], 0)","(posterior, a[1], 1)","(posterior, a[2], 2)","(posterior, a[3], 3)","(posterior, g[0], 0)","(posterior, g[1], 1)","(posterior, sigma)","(log_likelihood, score[0], 0)",...,"(sample_stats, mean_tree_accept)","(sample_stats, perf_counter_start)","(sample_stats, energy_error)","(sample_stats, step_size)","(sample_stats, max_energy_error)","(sample_stats, depth)","(sample_stats, tree_size)","(sample_stats, diverging)","(sample_stats, energy)","(sample_stats, process_time_diff)"
0,0,0,-1.130323,0.428683,-1.801091,-0.103095,0.236200,0.720884,0.425864,-0.070954,...,0.871617,1169.238632,0.401664,0.397527,0.820186,4,15.0,False,20.229142,0.001596
1,0,1,-1.664056,0.340213,-0.834904,0.478362,0.340148,0.723387,0.385726,-4.147343,...,0.956333,1169.240341,-0.479124,0.397527,-1.047678,2,3.0,False,17.178882,0.000467
2,0,2,-2.008341,-0.181530,-0.620523,-0.158988,0.735810,0.925338,0.786464,-3.085416,...,0.495141,1169.240921,0.846038,0.397527,1.266118,4,15.0,False,21.314292,0.001638
3,0,3,-1.065443,1.599116,-1.334224,0.305367,-0.219125,0.344952,0.559128,-0.342726,...,0.987480,1169.242676,-0.056466,0.397527,-0.426190,4,15.0,False,22.029038,0.001713
4,0,4,-1.466140,1.068115,-0.920037,0.881574,0.442463,-0.066000,0.654789,-1.991440,...,0.758809,1169.244531,0.272440,0.397527,0.511085,2,3.0,False,21.150677,0.000515


<IPython.core.display.Javascript object>

In [128]:
post_diff = (
    trace_m1_df[("posterior", "g[0]", 0)] - trace_m1_df[("posterior", "g[1]", 1)]
)

<IPython.core.display.Javascript object>

In [129]:
post_diff.mean()

-0.4223415186249017

<IPython.core.display.Javascript object>

In [130]:
post_diff.std()

0.5166181274773156

<IPython.core.display.Javascript object>

**Not sure if I set this up right and even if I did, I'm not sure what to do next.**

## Multi-level model

**Is this model right? I'm confused for what to put for the top line sigma for example**

$$\text{score}_i \text{ ~ Normal}(\mu_i, \sigma) $$
$$\mu_i = \alpha_{\text{SUBJECT}} + \gamma_{\text{TIME}} $$
$$\alpha_j \text{ ∼ Normal}(\bar{\alpha}, \sigma_\alpha) \tag{adaptive prior for subject j = 1..4}$$
$$\gamma_j \text{ ∼ Normal}(0, \sigma_\gamma) \tag{adaptive prior for time j = 1, 2}$$
$$\bar{\alpha} \text{ ∼ Normal}(0, 1.5) \tag{prior for average person}$$
$$\sigma_\alpha \text{ ~ Exp}(1) \tag{sigma for alpha}$$
$$\sigma_\gamma \text{ ~ Exp}(1) \tag{sigma for gamma}$$

In [134]:
with pm.Model() as m2:
    # Top line sigma (not sure if this is right)
    sigma = pm.Exponential("sigma", 1.0)

    # prior for average person and timepoint
    a_bar = pm.Normal("a_bar", 0.0, 1.5)

    # prior for SD of testers and time
    sigma_a = pm.Exponential("sigma_a", 1.0)
    sigma_g = pm.Exponential("sigma_g", 1.0)

    # adaptive priors?
    a = pm.Normal("a", a_bar, sigma_a, shape=4)  # 4 unique subjects
    g = pm.Normal("g", 0, sigma_g, shape=2)  # 2 time points

    # mu is deterministic, equivalent to alpha indexed by time
    mu = a[df_tests2["Name_code"]] + g[df_tests2["Time_code"]]

    # likelihood
    score = pm.Normal("score", mu=mu, sd=sigma, observed=df_tests2["Score_std"])

    trace_m2 = pm.sample(
        draws=1000, random_seed=19, return_inferencedata=True, progressbar=False
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [g, a, sigma_g, sigma_a, a_bar, sigma]
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 16 seconds.
There were 54 divergences after tuning. Increase `target_accept` or reparameterize.
There were 636 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.17202140484289688, but should be close to 0.8. Try to increase the number of tuning steps.
There were 64 divergences after tuning. Increase `target_accept` or reparameterize.
There were 99 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.632080573851144, but should be close to 0.8. Try to increase the number of tuning steps.
The rhat statistic is larger than 1.2 for some parameters.
The estimated number of effectiv

<IPython.core.display.Javascript object>

In [136]:
az.summary(trace_m2)

,mean,sd,hdi_5.5%,hdi_94.5%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
a_bar,-0.042,0.618,-0.997,0.882,0.046,0.033,177.0,177.0,122.0,738.0,1.04
a[0],-0.537,0.654,-1.590,0.357,0.048,0.050,187.0,85.0,115.0,409.0,1.07
a[1],0.474,0.714,-0.535,1.463,0.159,0.114,20.0,20.0,21.0,289.0,1.14
a[2],-0.471,0.632,-1.395,0.473,0.038,0.036,281.0,152.0,236.0,500.0,1.30
a[3],0.366,0.708,-0.567,1.341,0.173,0.125,17.0,17.0,18.0,640.0,1.16
g[0],-0.063,0.517,-0.701,0.660,0.074,0.053,49.0,49.0,26.0,332.0,1.11
g[1],-0.010,0.561,-0.644,0.797,0.124,0.089,20.0,20.0,17.0,6.0,1.18
sigma,0.951,0.439,0.445,1.594,0.139,0.101,10.0,10.0,13.0,134.0,1.24
sigma_a,0.758,0.570,0.097,1.426,0.155,0.112,14.0,14.0,9.0,4.0,1.39
sigma_g,0.566,0.497,0.036,1.082,0.024,0.017,424.0,424.0,164.0,71.0,1.26


<IPython.core.display.Javascript object>

Appendix: Environment and system parameters

In [123]:
%watermark -n -u -v -iv -w

Last updated: Wed Aug 04 2021

Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.20.0

seaborn   : 0.11.1
scipy     : 1.6.0
matplotlib: 3.3.4
arviz     : 0.11.1
daft      : 0.1.0
pandas    : 1.2.1
pymc3     : 3.11.0
json      : 2.0.9
numpy     : 1.20.1

Watermark: 2.1.0



<IPython.core.display.Javascript object>